## New York City Taxi Trip Duration

목차:

**[I. 데이터 불러오기 및 확인](#one)**

- [a. 데이터 불러오기 및 결측치 확인](#one-a)

**[II. EDA & FE](#two)**

- [a. 타겟 변수](#two-a)
- [b. 데이터 합치기](#two-b)
- [c. 날짜 변수](#two-c)
- [d. 거리 변수](#two-d)
- [e. 범주형 변수 원핫 인코딩](#two-e)
- [f. 상관 관계](#two-f)

**[III. 모델링](#three)**

- [a. train_test_split](#three-a)
- [b. RMSLE](#three-b)
- [c. 통계확인](#three-c)
- [d. lightgbm](#three-d)
- [e. xgboost](#three-e)

**[IV. 예측](#four)**

- [a. 앙상블](#four-a)

## I.데이터 불러오기 및 확인 <a id="one"></a>

1. 변수 | 설명
------- | -------
**id** | 각 여행에 대한 식별자
**vendor_id** | 여행 기록과 관련된 제공자를 나타내는 코드 
**pickup_datetime** | 미터가 작동 된 날짜 및 시간  
**dropoff_datetime** | 미터가 작동되지 않는 날짜 및 시간  
**passenger_count** | 차량의 승객 수 (운전자 입력 값) 
**pickup_longitude** | 미터가 사용 된 경도
**pickup_latitude** | 미터가 사용 된 위도 
**dropoff_longitude** | 미터가 사용 안 된 경도
**dropoff_latitude** | 미터가 사용 안 된 위도
**store_and_fwd_flag** | 이 플래그는 차량이 서버에 연결되어 있지 않기 때문에 공급 업체에 보내기 전에 여행 레코드가 차량 메모리에 보유되었는지 여부를 나타냅니다 (Y = 저장 및 전달, N = 저장 및 전달 여행이 아님).
**trip_duration** | 여행 시간 (초)

In [ ]:
import numpy as np
import pandas as pd 
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

### a. 데이터 불러오기 및 결측치 확인 <a id="one-a"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/nyc-taxi-trip-duration/train.zip')
test = pd.read_csv('/kaggle/input/nyc-taxi-trip-duration/test.zip')
sample_submission = pd.read_csv('/kaggle/input/nyc-taxi-trip-duration/sample_submission.zip')

In [ ]:
train.head(3)

In [ ]:
test.head(3)

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

>결측치 확인 결과 없는것으로 판단되었다.

## II. EDA & FE <a id="two"></a>

### a.타겟변수 확인 <a id="two-a"></a>

In [ ]:
from scipy import stats
from scipy.stats import norm

In [ ]:
plt.scatter(range(train.shape[0]),np.sort(train['trip_duration']))

산점도 확인 결과 이상치가 존재 하는것 같다

>train[train.trip_duration < train.trip_duration.quantile(0.99)] 유의수준1%에 해당하는 데이터 범위를 지정한뒤에 타겟변수를 설정해주고 살펴보았더니 전보다 편중이 덜 심해진 것을 볼 수 있다.

In [ ]:
sns.distplot(train.trip_duration.values, fit = norm)

>distplot 확인 결과 정규성을 띄지 않으므로 멱변환은 치해준다

In [ ]:
sns.distplot(np.log1p(train.trip_duration.values), fit = norm)

In [ ]:
train['trip_duration'] = np.log(train['trip_duration'].values)

>로그 변환 후 정규성을 띄는 것을 볼 수 있으므로 타겟변수인 trip_duration 피처에 대해서 로그변환을 시켜준다

### b.데이터 합치기 <a id="two-b"></a>

In [ ]:
feature_names=list(test)
df_train=train[feature_names]
df=pd.concat((df_train, test))

In [ ]:
print(train.shape, test.shape, df.shape)

> 데이터 전처리를 편리하게 하기 위해서 train데이터와 test 데이터를 합쳤다.

In [ ]:
df.head(3)

### c.날짜 변수 <a id="two-c"></a>

In [ ]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [ ]:
df['month'] = df['pickup_datetime'].dt.month
df['day'] = df['pickup_datetime'].dt.day
df['weekday'] = df['pickup_datetime'].dt.weekday
df['hour'] = df['pickup_datetime'].dt.hour
df['dayofweek'] = df['pickup_datetime'].dt.dayofweek

In [ ]:
df.drop(['pickup_datetime'], axis=1, inplace=True)

In [ ]:
sns.countplot(df['hour'])

> 픽업이 새벽에는 매우 낮고 6시~8시 사이에 제일 높은것을 알 수 있다

In [ ]:
sns.countplot(df['dayofweek'])

> 픽업이 월요일에 제일 낮은것을 볼 수 있다.

### d.거리 변수 <a id="two-d"></a>

In [ ]:
df['dist_long'] = df['pickup_longitude'] - df['dropoff_longitude']
df['dist_lat'] = df['pickup_latitude'] - df['dropoff_latitude']

In [ ]:
df['dist'] = np.sqrt(np.square(df['dist_long']) + np.square(df['dist_lat']))

>미터기가 시작되고 끝난 각각의 경도 위도만큼 빼준다음에 총 합계를 구해서 새로운 파생변수를 만들었다

하버사인 공식(Haversine Formula) 은 주어진 지점에 대해 구 (Sphere) 의 두 지점 사이의 최단거리(great-circle distance) 를 구하는 공식이다.

$$ 2rarcsin\sqrt{\sin^2(\frac{φ_2 -φ_1}{2}) + cos(φ_1)cos(φ_2)sin^2(\frac{λ_2 -λ_1}{2})} $$

In [ ]:
def ft_haversine_distance(lat1, lng1, lat2, lng2):
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371 #km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return h

df['distance'] = ft_haversine_distance(df['pickup_latitude'].values,
                                       df['pickup_longitude'].values, 
                                       df['dropoff_latitude'].values,
                                       df['dropoff_longitude'].values)

>두 위도와 경도 사이의 거리를 구하는 공식으로 하버사인 공식을 사용해서 거리 피처를 구해줍니다.

In [ ]:
df.boxplot(column='distance')

In [ ]:
#df = df[(df.distance < 200)]

>boxplot 확인 결과 이상치 제거를 해줍니다.

In [ ]:
g_vendor = train.groupby('vendor_id')['trip_duration'].mean()
sns.barplot(g_vendor.index,g_vendor.values)

> 공급업체 별로 그룹바이를 한 상태에서 타겟변수를 확인해본 결과 별 차이는 없는것같다

In [ ]:
sfflag = train.groupby('store_and_fwd_flag')['trip_duration'].mean()
sns.barplot(sfflag.index,sfflag.values)

>'store_and_fwd_flag'는 여행 시간을 잘 구별하는 것처럼 보인다

In [ ]:
pc = train.groupby('passenger_count')['trip_duration'].mean()
sns.barplot(pc.index,pc.values)

### e. 범주형 변수 원핫 인코딩<a id="two-e"></a>

In [ ]:
df = pd.concat([df, pd.get_dummies(df['store_and_fwd_flag'],prefix = 'store')], axis=1)
df.drop(['store_and_fwd_flag'], axis=1, inplace=True)

df = pd.concat([df, pd.get_dummies(df['vendor_id'],prefix = 'vendor')], axis=1)
df.drop(['vendor_id'], axis=1, inplace=True)

In [ ]:
df.head(3)

> 범주형 변수들을 원 핫 인코딩을 이용하여 범주화 해줍니다.

### f. 상관 관계<a id="two-f"></a>

In [ ]:
cor = df.corr()
mask = np.array(cor)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(cor,mask= mask,square=True,annot=True)

## III. 모델링 <a id="three"></a>

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

### a. train_test_split <a id="three-a"></a>

In [ ]:
df.head(3)

In [ ]:
df.drop(["id"], axis=1, inplace=True)

In [ ]:
new_train = df[:train.shape[0]]
new_test = df[train.shape[0]:]

In [ ]:
target = train['trip_duration']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(new_train, target, test_size=0.2, shuffle=True)

> df 데이터를 전처리를 끝마친 다음에 다시 데이터를 나누어 주었으며 검증을 위해서 trian_test_split을 하였고 gridsearchcv를 통해서 모델링을 할때 검증데이터로 8:2로 나누었습니다.

### b. RMSLE <a id="three-b"></a>

Submissions are evaluated one the Root Mean Squared Logarithmic Error (RMSLE)

$$ \sqrt{\frac{1}{n} \sum_{i=1}^n (\log(p_i + 1) - \log(a_i+1))^2 } $$
${n}$ is the number of hours in the test set
$p_i$ is your predicted count
$a_i$ is the actual count
$\log(x)$ is the natural logarithm

In [ ]:
def rmsle_score(preds, true):
    rmsle_score = (np.sum((np.log1p(preds)-np.log1p(true))**2)/len(true))**0.5
    return rmsle_score

In [ ]:
from sklearn.metrics.scorer import make_scorer

RMSLE = make_scorer(rmsle_score)

> rmsle score를 만들어주어서 모델을 비교할때 평가치로 사용합니다

### c. 통계 확인 <a id="three-c"></a>

In [ ]:
import statsmodels.api as sm

In [ ]:
model = sm.OLS(target.values, new_train.astype(float))

In [ ]:
re = model.fit()
re.summary()

>모델의 설명력이 0.338이며 각 피처의 p-value가 0.0000으로 유의수준 5%보다 낮았으며 
다중공선성의 문제는 없는것으로 보인다

### d. lightgbm <a id="three-d"></a>

In [ ]:
import lightgbm as lgbm

In [ ]:
lgb_params = {
    'metric' : 'rmse',
    'learning_rate': 0.1,
    'max_depth': 25,
    'num_leaves': 1000, 
    'objective': 'regression',
    'feature_fraction': 0.9,
    'bagging_fraction': 0.5,
    'max_bin': 1000 }


In [ ]:
lgb_df = lgbm.Dataset(new_train,target)

In [ ]:
lgb_model = lgbm.train(lgb_params, lgb_df, num_boost_round=1500)

In [ ]:
pred = lgb_model.predict(new_test)

In [ ]:
pred_lgb = np.exp(pred)

> lightgbm 모델을 이용하여서 학습시킨다음에 예측치를 구해줍니다

### e. xgboost <a id="three-d"></a>

In [ ]:
import xgboost as xgb

In [ ]:
params = {
    'booster':            'gbtree',
    'objective':          'reg:linear',
    'learning_rate':      0.1,
    'max_depth':          14,
    'subsample':          0.8,
    'colsample_bytree':   0.7,
    'colsample_bylevel':  0.7,
    'silent':             1
}

In [ ]:
dtrain = xgb.DMatrix(new_train, target)

In [ ]:
gbm = xgb.train(params,
                dtrain,
                num_boost_round = 200)

In [ ]:
pred_test = np.exp(gbm.predict(xgb.DMatrix(new_test)))

> xgboost 모델을 이용하여서 학습시킨다음에 예측치를 구해줍니다

## IV. 예측 <a id="four"></a>

### a.앙상블 <a id="four-a"></a>

In [ ]:
#ensemble = (0.8*pred_lgb + 0.4*pred_test) 0.42295
#ensemble = (0.7*pred_lgb + 0.3*pred_test) 0.38148
ensemble = (0.6*pred_lgb + 0.4*pred_test) #0.38124
#ensemble = (0.55*pred_lgb + 0.45*pred_test) 0.38126

In [ ]:
sub = pd.DataFrame()
sub['id'] = test.id
sub['trip_duration'] = ensemble
sub.head(3)

In [ ]:
sub = pd.DataFrame()
sub['id'] = test.id
sub['trip_duration'] = ensemble
sub.head(3)

In [ ]:
sub.to_csv('submission.csv', index=False)

> 앙상블을 통해서 lgb와 xgb를 6:4 비율로 한다음에 submission 파일을 만들어 제출합니다.